# Time to Build

## Connect to Weaviate

In [ ]:
import weaviate, os

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"), # Replace with your inference API key
    }
)

In [ ]:
import weaviate, os

# Connect to a cloud instance of Weaviate (with WCS)
client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"), # Replace with your inference API key
    }
)

client.is_ready()

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print(item.properties)

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
import weaviate.classes.query as wq
questions = client.collections.get("Questions")

response = questions.query.near_text(
    query="musical instruments",
    limit=5,
    filters=wq.Filter.by_property("value").greater_than(500)
)

for item in response.objects:
    print(item.properties)

In [ ]:
from datetime import datetime, timezone
import weaviate.classes.query as wq
questions = client.collections.get("Questions")

response = questions.query.near_text(
    query="musical instruments",
    limit=5,
    filters= # in the year 2000
        wq.Filter.by_property("airDate").greater_than(datetime(2000, 1, 1).replace(tzinfo=timezone.utc)) &
        wq.Filter.by_property("airDate").less_than(datetime(2001, 1, 1).replace(tzinfo=timezone.utc))
        # filters=wq.Filter.by_property("airDate").greater_than("2000-01-01T00:00:00+00:00")
)

for item in response.objects:
    print(item.properties)

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
)

for item in response.objects:
    print(item.properties)

## Retrieval Augmented Generation!!!
[Docs - RAG](https://weaviate.io/developers/weaviate/search/generative)

In [ ]:
questions = client.collections.get("Questions")

response = questions.generate.near_text(
    query="musical instruments",
    limit=4,
    single_prompt="Write a short tweet about: {question}"
)

for item in response.objects:
    print(item.properties)
    print(item.generated)


In [ ]:
questions = client.collections.get("Questions")

response = questions.generate.near_text(
    query="musical instruments",
    limit=4,
    grouped_task="Explain what this content is about."
)

print (response.generated)